# Homework 2

In [40]:
# necessary import statements
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
# from plotnine import *
# import matplotlib.pyplot as plt

from sklearn import metrics 
from sklearn.linear_model import LogisticRegression # Logistic Regression Model
from sklearn.preprocessing import StandardScaler #Z-score variables
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score

from sklearn.tree import DecisionTreeClassifier # Decision Tree
from sklearn.neighbors import KNeighborsClassifier # KNN

from sklearn.model_selection import train_test_split # simple TT split cv
from sklearn.model_selection import KFold # k-fold cv

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [41]:
# importing the data into a DataFrame
WordRecall = pd.read_csv("https://raw.githubusercontent.com/cmparlettpelleriti/CPSC392ParlettPelleriti/master/Data/HW2.csv")
WordRecall = WordRecall.dropna()
WordRecall.head(15)

,Position,correct,WordList,Trial.Number,out_degree,in_degree,Length,Log_Freq_HAL,Ortho_N,Phono_N,Concreteness_Rating,Age_Of_Acquisition,Female,State
0,4,1,0,106,10.0,2.0,3.0,8.590,14.0,20.0,3.52,11.05,1,NM
1,1,0,1,37,8.0,3.0,4.0,7.920,8.0,33.0,4.14,9.12,0,CA
2,2,0,0,56,9.0,2.0,4.0,8.449,16.0,21.0,3.52,8.84,1,WA
3,1,0,0,85,11.0,14.0,4.0,10.800,9.0,1.0,1.87,8.39,1,WA
4,1,0,1,115,19.0,14.0,5.0,8.158,2.0,9.0,5.00,8.47,1,CA
5,1,1,0,61,16.0,3.0,4.0,7.591,13.0,28.0,3.86,9.72,1,NM
6,3,0,0,75,11.0,13.0,4.0,8.719,5.0,11.0,3.27,6.83,1,NM
7,2,1,1,104,12.0,22.0,5.0,8.492,0.0,2.0,2.93,8.11,1,CA
8,3,0,0,111,11.0,2.0,4.0,8.350,14.0,13.0,4.00,6.94,1,NM
11,4,1,1,64,10.0,3.0,4.0,8.331,8.0,12.0,3.46,11.05,1,NM


### 1. Build a KNN, Decision Tree, AND Logistic Regression model to predict whether or not someone recalled a word (correct) using all the other variables.

In [42]:
 # If a categorical variable has more than 2 levels, use get_dummies() to create dummy variables for it (covered in All The Stuff You Need to Know Python)

 # Position and State are factor variables, so we will get dummies for these columns
dummies = pd.get_dummies(WordRecall, columns = ["Position", "State"])
WordRecall = dummies
WordRecall.columns

Index(['correct', 'WordList', 'Trial.Number', 'out_degree', 'in_degree',
       'Length', 'Log_Freq_HAL', 'Ortho_N', 'Phono_N', 'Concreteness_Rating',
       'Age_Of_Acquisition', 'Female', 'Position_1', 'Position_2',
       'Position_3', 'Position_4', 'Position_5', 'Position_6', 'State_CA',
       'State_ID', 'State_NM', 'State_OR', 'State_WA'],
      dtype='object')

In [43]:
predictors = ['WordList', 'Trial.Number', 'out_degree', 'in_degree',
       'Length', 'Log_Freq_HAL', 'Ortho_N', 'Phono_N', 'Concreteness_Rating',
       'Age_Of_Acquisition', 'Female', 'Position_1', 'Position_2',
       'Position_3', 'Position_4', 'Position_5', 'Position_6', 'State_CA',
       'State_ID', 'State_NM', 'State_OR', 'State_WA']
cont_predictors = ['WordList', 'Trial.Number', 'out_degree', 'in_degree',
       'Length', 'Log_Freq_HAL', 'Ortho_N', 'Phono_N', 'Concreteness_Rating',
       'Age_Of_Acquisition']
X = WordRecall[predictors]
y = WordRecall["correct"]
WordRecall.head()

,correct,WordList,Trial.Number,out_degree,in_degree,Length,Log_Freq_HAL,Ortho_N,Phono_N,Concreteness_Rating,...,Position_2,Position_3,Position_4,Position_5,Position_6,State_CA,State_ID,State_NM,State_OR,State_WA
0,1,0,106,10.0,2.0,3.0,8.590,14.0,20.0,3.52,...,0,0,1,0,0,0,0,1,0,0
1,0,1,37,8.0,3.0,4.0,7.920,8.0,33.0,4.14,...,0,0,0,0,0,1,0,0,0,0
2,0,0,56,9.0,2.0,4.0,8.449,16.0,21.0,3.52,...,1,0,0,0,0,0,0,0,0,1
3,0,0,85,11.0,14.0,4.0,10.800,9.0,1.0,1.87,...,0,0,0,0,0,0,0,0,0,1
4,0,1,115,19.0,14.0,5.0,8.158,2.0,9.0,5.00,...,0,0,0,0,0,1,0,0,0,0


In [44]:
# Use the train_test_split() to do an 90/10 split (make sure to use the SAME split when training all 3 models, do not re-split your data. We want each model to be trained on the same training set).
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.1)
X_train

,WordList,Trial.Number,out_degree,in_degree,Length,Log_Freq_HAL,Ortho_N,Phono_N,Concreteness_Rating,Age_Of_Acquisition,...,Position_2,Position_3,Position_4,Position_5,Position_6,State_CA,State_ID,State_NM,State_OR,State_WA
2236,0,83,9.0,20.0,4.0,7.328,1.0,8.0,3.15,7.28,...,0,0,0,1,0,0,0,1,0,0
5968,0,68,15.0,2.0,7.0,5.948,0.0,3.0,3.77,8.17,...,1,0,0,0,0,0,0,0,0,1
9584,0,49,8.0,52.0,5.0,9.330,4.0,10.0,3.84,6.17,...,0,0,0,0,0,0,0,0,0,1
9497,0,2,13.0,2.0,5.0,7.264,3.0,8.0,3.93,7.72,...,1,0,0,0,0,0,1,0,0,0
1514,0,101,18.0,14.0,3.0,9.062,18.0,23.0,4.29,7.05,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3037,0,43,17.0,6.0,4.0,8.582,9.0,13.0,2.48,8.05,...,0,0,0,0,0,0,0,1,0,0
14629,0,84,13.0,7.0,5.0,7.728,5.0,13.0,4.19,6.00,...,0,0,0,0,1,1,0,0,0,0
8328,1,83,10.0,9.0,4.0,8.333,11.0,15.0,4.21,4.89,...,0,0,0,1,0,0,0,0,1,0
13992,0,79,13.0,48.0,4.0,9.813,4.0,27.0,4.97,6.67,...,0,0,0,0,0,1,0,0,0,0


In [45]:
# Appropriately z-score your continuous/interval variables only

In [46]:
# For KNN, include only continuous/interval columns as predictors. For Decision Tree and Logistic Regression use ALL columns (other than correct).
# For KNN, choose K by using GridSearchCV and an appropriate range of potential K's.
# For Decision Trees, use GridSearchCV and an appropriate range of numbers to choose min_samples_leaf.

In [47]:
# KNN


knn = KNeighborsClassifier()

# create z score object
z = make_column_transformer((StandardScaler(), cont_predictors),
                            remainder = "passthrough")

# make pipeline
pipe = make_pipeline(z, knn)

# choose potential values of k
ks = {"kneighborsclassifier__n_neighbors": range(1,30)}

# use grid search to find best parameters
grid1 = GridSearchCV(pipe, ks, scoring = "accuracy", cv = 5, refit = True)

grid1.fit(X_train[cont_predictors], y_train)

print(grid1.best_estimator_.get_params()["kneighborsclassifier__n_neighbors"])

1


In [48]:
#metrics
print("KNN Results: ")
print("------------------------")
print("------------------------")
print("------------------------")
print("TEST Acc: ", accuracy_score(y_test, grid1.predict(X_test)))
print("TRAIN Acc: ", accuracy_score(y_train, grid1.predict(X_train)))

print("TEST Precision : ", precision_score(y_test, grid1.predict(X_test)))
print("TRAIN Precision: ", precision_score(y_train, grid1.predict(X_train)))

print("TEST Recall : ", recall_score(y_test, grid1.predict(X_test)))
print("TRAIN Recall: ", recall_score(y_train, grid1.predict(X_train)))

print("TEST ROC/AUC : ", roc_auc_score(y_test, grid1.predict_proba(X_test)[:,1]))
print("TRAIN ROC/AUC: ", roc_auc_score(y_train, grid1.predict_proba(X_train)[:,1]))

KNN Results: 
------------------------
------------------------
------------------------
TEST Acc:  0.715312724658519
TRAIN Acc:  0.9165601023017903
TEST Precision :  0.7250673854447439
TRAIN Precision:  0.9211800968736239
TEST Recall :  0.736986301369863
TRAIN Recall:  0.9252543122512162
TEST ROC/AUC :  0.7141815016682901
TRAIN ROC/AUC:  0.9157603381818133


In [49]:
'''
z3 = StandardScaler()
X_train[cont_predictors] = z3.fit_transform(X_train[cont_predictors])
X_test[cont_predictors] = z3.transform(X_test[cont_predictors])

knn2 = KNeighborsClassifier(n_neighbors = 4)

knn2.fit(X_train[cont_predictors], y_train)
'''

'\nz3 = StandardScaler()\nX_train[cont_predictors] = z3.fit_transform(X_train[cont_predictors])\nX_test[cont_predictors] = z3.transform(X_test[cont_predictors])\n\nknn2 = KNeighborsClassifier(n_neighbors = 4)\n\nknn2.fit(X_train[cont_predictors], y_train)\n'

In [50]:
#metrics
'''
print("KNN Results: ")
print("------------------------")
print("------------------------")
print("------------------------")
print("TEST Acc: ", accuracy_score(y_test, knn2.predict(X_test[cont_predictors])))
print("TRAIN Acc: ", accuracy_score(y_train, knn2.predict(X_train[cont_predictors])))

print("TEST Precision : ", precision_score(y_test, knn2.predict(X_test[cont_predictors])))
print("TRAIN Precision: ", precision_score(y_train, knn2.predict(X_train[cont_predictors])))

print("TEST Recall : ", recall_score(y_test, knn2.predict(X_test[cont_predictors])))
print("TRAIN Recall: ", recall_score(y_train, knn2.predict(X_train[cont_predictors])))

print("TEST ROC/AUC : ", roc_auc_score(y_test, knn2.predict_proba(X_test[cont_predictors])[:,1]))
print("TRAIN ROC/AUC: ", roc_auc_score(y_train, knn2.predict_proba(X_train[cont_predictors])[:,1]))
'''

'\nprint("KNN Results: ")\nprint("------------------------")\nprint("------------------------")\nprint("------------------------")\nprint("TEST Acc: ", accuracy_score(y_test, knn2.predict(X_test[cont_predictors])))\nprint("TRAIN Acc: ", accuracy_score(y_train, knn2.predict(X_train[cont_predictors])))\n\nprint("TEST Precision : ", precision_score(y_test, knn2.predict(X_test[cont_predictors])))\nprint("TRAIN Precision: ", precision_score(y_train, knn2.predict(X_train[cont_predictors])))\n\nprint("TEST Recall : ", recall_score(y_test, knn2.predict(X_test[cont_predictors])))\nprint("TRAIN Recall: ", recall_score(y_train, knn2.predict(X_train[cont_predictors])))\n\nprint("TEST ROC/AUC : ", roc_auc_score(y_test, knn2.predict_proba(X_test[cont_predictors])[:,1]))\nprint("TRAIN ROC/AUC: ", roc_auc_score(y_train, knn2.predict_proba(X_train[cont_predictors])[:,1]))\n'

In [51]:
# Decision Tree
tree = DecisionTreeClassifier()

# make pipe
pipe = make_pipeline(z,tree)

# choose potential values of max depths
min_leaf_sample_dict = {"decisiontreeclassifier__min_samples_leaf": range(1,9)}

# use grid search to find best parameters
grid2 = GridSearchCV(pipe, min_leaf_sample_dict, scoring = "accuracy", cv = 5, refit = True)

grid2.fit(X_train, y_train)


# min_leaf_sample_dict = {"min_samples_leaf": range(6,15)}

# grid2 = GridSearchCV(tree, min_leaf_sample_dict, scoring = 'accuracy', cv = 5, refit = True)

# grid2.fit(X_train, y_train)

# tree = DecisionTreeClassifier(min_samples_leaf = grid2.best_estimator_.get_params()["min_samples_leaf"])
# tree.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('standardscaler',
                                                                         StandardScaler(),
                                                                         ['WordList',
                                                                          'Trial.Number',
                                                                          'out_degree',
                                                                          'in_degree',
                                                                          'Length',
                                                                          'Log_Freq_HAL',
                                                                          'Ortho_N',
                                                                          'Phono_N',
                                                                          'Concreteness_Rating',
                                                                          'Age_Of_Acquisition'])])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__min_samples_leaf': range(1, 9)},
             scoring='accuracy')

In [52]:
print(grid2.best_estimator_.get_params()["decisiontreeclassifier__min_samples_leaf"])

1


In [53]:
#metrics
print("Decision Tree Results: ")
print("------------------------")
print("------------------------")
print("------------------------")
print("TEST Acc: ", accuracy_score(y_test, grid2.predict(X_test)))
print("TRAIN Acc: ", accuracy_score(y_train, grid2.predict(X_train)))

print("TEST Precision : ", precision_score(y_test, grid2.predict(X_test)))
print("TRAIN Precision: ", precision_score(y_train, grid2.predict(X_train)))

print("TEST Recall : ", recall_score(y_test, grid2.predict(X_test)))
print("TRAIN Recall: ", recall_score(y_train, grid2.predict(X_train)))

print("TEST ROC/AUC : ", roc_auc_score(y_test, grid2.predict_proba(X_test)[:,1]))
print("TRAIN ROC/AUC: ", roc_auc_score(y_train, grid2.predict_proba(X_train)[:,1]))

Decision Tree Results: 
------------------------
------------------------
------------------------
TEST Acc:  0.7498202731847592
TRAIN Acc:  0.9880914322250639
TEST Precision :  0.7616438356164383
TRAIN Precision:  0.9970032963739887
TEST Recall :  0.7616438356164383
TRAIN Recall:  0.9809818664307829
TEST ROC/AUC :  0.7520713738005927
TRAIN ROC/AUC:  0.9997035626039007


In [54]:
# Logistic model

z2 = StandardScaler()
z2.fit(X_train[cont_predictors])

X_train[cont_predictors] = z2.transform(X_train[cont_predictors])
X_test[cont_predictors] = z2.transform(X_test[cont_predictors])


myLogit = LogisticRegression(penalty = 'none')
myLogit.fit(X_train, y_train)
testPredictedVals = myLogit.predict(X_test)
trainPredictedVals = myLogit.predict(X_train)

In [55]:
#metrics
print("Logistic Model Results: ")
print("------------------------")
print("------------------------")
print("------------------------")
print("TEST Acc: ", accuracy_score(y_test, testPredictedVals))
print("TRAIN Acc: ", accuracy_score(y_train, trainPredictedVals))

print("TEST Precision : ", precision_score(y_test, testPredictedVals))
print("TRAIN Precision: ", precision_score(y_train, trainPredictedVals))

print("TEST Recall : ", recall_score(y_test, testPredictedVals))
print("TRAIN Recall: ", recall_score(y_train, trainPredictedVals))

print("TEST ROC/AUC : ", roc_auc_score(y_test, myLogit.predict_proba(X_test)[:,1]))
print("TRAIN ROC/AUC: ", roc_auc_score(y_train, myLogit.predict_proba(X_train)[:,1]))

Logistic Model Results: 
------------------------
------------------------
------------------------
TEST Acc:  0.6542056074766355
TRAIN Acc:  0.6621643222506394
TEST Precision :  0.6640316205533597
TRAIN Precision:  0.6824150727947473
TEST Recall :  0.6904109589041096
TRAIN Recall:  0.7048503611971104
TEST ROC/AUC :  0.7024122852465132
TRAIN ROC/AUC:  0.7128366978250819


### 2.
### A) Using the performance metrics you printed in  part 1 thoroughly discuss which model did best (if you had to pick one), how can you tell? (make sure to contextualize the metrics you discuss instead of just saying "it has a good sensitivity". Include a discussion of whether the model is overfit, underfit, or neither.

#### If I had to pick a model, I would choose either the KNN or the decision tree model because the logit model has worse performance metrics than both of the other models. Between the KNN and the decision tree models, I think the decision tree is a slightly better model because it has better performance metrics across the train and test sets (except for the test set precision metric). I like using the ROC AUC as the most important metric because it tells us how well the model distinguishes between positive and negative cases, and the decision tree has the best ROC AUC for the train and the test sets.

### B)
### Discuss whether, based on this performance, you'd "put this model into production" (e.g. use it and trust it to predict future data). In your discussion be sure to consider the performance of this specific model (as discussed in A) as well as the context of this specific situation (is this making a high stakes prediction? Are false positives more important than false negatives or are they the same?).

#### We are predicting whether people will remember a word based on characteristics of the word itself and the list it is a part of. I would put this decision tree model into production because the consequences of predicting false negatives or false positives are not severe, and it performs decently across all performance metrics. False positives and false negatives are equally important. A false positive would be predicting that a person recalled a word, but they didn't. A false negative would be predicting that a person didn't recall a word, but they actually did. This model has approximately 75% accuracy, precision, recall, and ROC AUC on the test set, so it is better than a random guess, and better than our KNN and logistic models.

In [59]:
# doesn't show this cells output when downloading PDF
!pip install gwpy &> /dev/null

# installing necessary files
!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
!sudo apt-get update
!sudo apt-get install texlive-xetex texlive-fonts-recommended texlive-plain-generic

# installing pypandoc
!pip install pypandoc

# connecting your google drive
from google.colab import drive
drive.mount('/content/drive')

# copying your file over. Change "Homework2.ipynb" to whatever your file is called (see top of notebook)
!cp "drive/My Drive/Colab Notebooks/CPSC392/Homework2.ipynb" ./

# Again, replace "Homework2.ipynb" to whatever your file is called (see top of notebook)
!jupyter nbconvert --to PDF "Homework2.ipynb"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pandoc is already the newest version (2.5-3build2).
texlive is already the newest version (2019.20200218-1).
texlive-latex-extra is already the newest version (2019.202000218-1).
texlive-xetex is already the newest version (2019.20200218-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/d